In [ ]:
import pathlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
from Orchid.FractureDiagnostics.RatioTypes import ProppantConcentration, SlurryRate
import UnitsNet

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
    'permian': 'Project_frankNstein_Permian_UTM13_FEET.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
@toolz.curry
def to_project_unit(unit, nq):
    return nq.ToUnit(unit)

@toolz.curry
def net_quantity_to_tuple(nq):
    return nq.Value, nq.Unit

def project_fluid_density(proj):
    magnitude, unit = toolz.pipe(proj.FluidDensity,
                                 to_project_unit(proj.ProjectUnits.DensityUnit),
                                 net_quantity_to_tuple,
                                )
    return f'{magnitude} {UnitsNet.Density.GetAbbreviation(unit)}'

def project_azimuth(proj):
    magnitude, unit = net_quantity_to_tuple(proj.Azimuth)
    return f'{magnitude} {UnitsNet.Angle.GetAbbreviation(unit)}'

def project_center(proj):
    result = toolz.pipe(proj.GetProjectCenter(),
                        toolz.map(to_project_unit(proj.ProjectUnits.LengthUnit)),
                        toolz.map(lambda m: f'{m.Value} {UnitsNet.Length.GetAbbreviation(m.Unit)}'),
                        tuple,
                       )
    return result

In [ ]:
def get_project_measurements(proj):
    result = {
        'fluid_density': project_fluid_density(proj),
        'azimuth': project_azimuth(proj),
        'center_x': project_center(proj)[0],
        'center_y': project_center(proj)[1],
    }
    return result

In [ ]:
def collect_project_measurements(e):
    field, measurements = e
    field_summary = toolz.merge(measurements, {'field': field})
    return field, field_summary

project_measurements = toolz.pipe(
                            projects,
                            toolz.valmap(get_project_measurements),
                            toolz.itemmap(collect_project_measurements),
                        )

In [ ]:
def make_data_frame(field):
    return pd.DataFrame(data=project_measurements[field], 
                        columns=['field', 'fluid_density', 'azimuth', 'center_x', 'center_y'],
                        index=(0,))

In [ ]:
make_data_frame('bakken')

In [ ]:
make_data_frame('montney')

In [ ]:
make_data_frame('permian')

In [ ]:
str(projects['bakken'].TimeZone)

In [ ]:
time_zones = toolz.valmap(lambda p: str(p.TimeZone), projects)
time_zones

In [ ]:
toolz.pipe(projects['bakken'].GetProjectBounds(),
           toolz.map(lambda l: f'{l.Value:10} {UnitsNet.Length.GetAbbreviation(l.Unit)}'),
           list)

In [ ]:
def length_abbreviation(l):
    return f'{UnitsNet.Length.GetAbbreviation(l.Unit)}'

def length_magnitude(l):
    return f'{l.Value:10}'

def str_length(l):
    return f'{length_magnitude(l)} {length_abbreviation(l)}'

def get_project_bounds(proj):
    min_x, max_x, min_y, max_y, min_depth, max_depth = proj.GetProjectBounds()
    magnitude_format = "G10"
    result = {
        'min_x': str_length(min_x),
        'max_x': str_length(max_x),
        'min_y': str_length(min_y),
        'max_y': str_length(max_y),
        'min_depth': str_length(min_depth),
        'max_depth': str_length(max_depth),
    }
    return result

In [ ]:
def collect_project_bounds(e):
    field, measurements = e
    field_summary = toolz.merge(measurements, {'field': field})
    return field, field_summary

project_bounds = toolz.pipe(
    projects,
    toolz.valmap(get_project_bounds),
    toolz.itemmap(collect_project_bounds),
)

In [ ]:
def make_bounds_data_frame(field):
    return pd.DataFrame(data=project_bounds[field], 
                        columns=['field', 'min_x', 'max_x', 'min_y', 'max_y', 'min_depth', 'max_depth'],
                        index=(0,))

In [ ]:
make_bounds_data_frame('bakken')

In [ ]:
make_bounds_data_frame('montney')

In [ ]:
make_bounds_data_frame('permian')